<a href="https://colab.research.google.com/github/Lev-Stambler/ECCT-Evaluator/blob/main/FunctionalVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import copy
import numbers
from sympy import *
import pdb
import multiprocessing
import numpy as np

class ScenarioGraph:

  # Input: 'scenarios' is a list of scenarios,
  # 'edges' is an adjacency list from scenario indices (even scenarios without outgoing edges) to
  # lists of child scenario indices (potentially empty),
  # 'weight' is a map from edges to probability values, possibly symbolic.
  # 'choice_map' is a map from scenario indices to {-1, 0, 1}. A value of 0 indicates a scenarios
  # is not a choice. A value of 1 indicates a scenario is a choice and the offensive team chooses.
  # A value of -1 indicates a scenario is a choice and the defensive team chooses.
  def __init__(self, scenarios, edges, weight, choice_map):
      self.scenarios = scenarios
      self.edges = edges
      self.weight = weight
      self.choice_map = choice_map

      # Checks that the events of scenarios form a partition
      # List of all events
      lst_events = []

      # Set of all events
      set_events = set([])

      # Compares whether the list and the set are the same size.
      # because in a partition no element should be repeated and
      # all equivalence classes should be disjoint.
      for scenario in scenarios:
          for (event, i) in scenario.keys():
              if i == 0:
                  lst_events.append(event)
                  set_events.add(event)

      if len(lst_events) != len(set_events):
          raise TypeError("scenario events don't form  a partition")

      # Checks that weight map is one-to-one
      if len(weight.values()) != len(weight.keys()):
          raise TypeError("weight map not injective!")

      # Checks that choice map only takes values -1, 0, 1
      if not set(choice_map.values()).issubset({-1, 0, 1}):
          raise TypeError("not valid choice map")

class Condition:
    # A condition gives a single variable symbolic expression in terms
    # of the variable 'x' (one for each numerical state
    # parameter, score, team with possession, and time remaining) that
    # the parameter can either satisfy or not. Note we do not give
    # the flexibility for an event condition because the assumption is
    # if an event should not happen after another state, the possibility
    # would not be programmed into a scenario diagram. When no
    # condition is to be placed on a paramter an empty string will be
    # the placeholder.
    def __init__(self, score_cond, team_cond, time_cond):
        self.score_cond = score_cond
        self.team_cond = team_cond
        self.time_cond = time_cond

    # Returns whether the state satsifies the conditions.
    def satisfies(self, state):
        score_dif = state[0]
        team = state[1]
        time_rem = state[3]

        # For each case returns whether the state satisfied the non-empty conditions.
        # when all conditions are empty returns false.
        x = symbols('x')

        if (self.score_cond == '') and (self.team_cond == '') and (self.time_cond == ''):
            return False
        elif (self.score_cond != '') and (self.team_cond == '') and (self.time_cond == ''):
            return self.score_cond.subs('x', score_dif)
        elif (self.score_cond == '') and (self.team_cond != '') and (self.time_cond == ''):
            return self.team_cond.subs('x', team)
        elif (self.score_cond == '') and (self.team_cond == '') and (self.time_cond != ''):
            return self.time_cond.subs('x', time_rem)
        elif (self.score_cond != '') and (self.team_cond != '') and (self.time_cond == ''):
            return ((self.score_cond.subs('x', score_dif)) and (self.team_cond.subs('x', team)))
        elif (self.score_cond != '') and (self.team_cond == '') and (self.time_cond != ''):
            return ((self.score_cond.subs('x', score_dif)) and (self.time_cond.subs('x', time_rem)))
        elif (self.score_cond == '') and (self.team_cond != '') and (self.time_cond != ''):
            return ((self.team_cond.subs('x', team)) and (self.time_cond.subs('x', time_rem)))
        else:
            return ((self.score_cond.subs('x', score_dif)) and (self.team_cond.subs('x', team)) and (self.time_cond.subs('x', time_rem)))


class Conditions:

    # 'conditions' is a list of condition objects.
    def __init__(self, conditions):
        self.conditions = conditions

    # Iterates through the conditions and if the input
    # 'state' satisfies and of them the function returns true.
    def satisfies(self, state):
        for condition in self.conditions:
            if condition.satisfies(state):
                return True
        return False

class LimitingCondition:
    # 'restrictions' is a dictionary from Conditions objects to list of events.
    def __init__(self, restrictions):
        self.restrictions = restrictions

    # Input is a state. Output is a dictionary of all the forbidden events.
    def forbidden_event(self, state):
        all_events = []
        for conditions, events in self.restrictions.items():
            if conditions.satisfies(state):
                all_events.extend(events)
        return all_events

# Input: a scenario graph and a dictionary which maps
# probability symbols to numeric values. Outputs a new scenario graph
# in which all the probability symbols are replaced with their
# numeric value.
def numericalScenario(scen_graph, probAssign):
    # The new scenario graph has the same edges, weight map, and choice map
    # so it suffices to simply replace the scenarios.
    new_scenarios = []

    # Stores a map from index of scenarios with symbols to their
    # updated scenario with the symbol replaced.
    for scen in scen_graph.scenarios:
        # Copies scenario but replaces symbol with number.
        newScen = {}

        for (key, value) in scen.items():
            newKey = copy.deepcopy(key)
            newValue = copy.deepcopy(value)

            # need in order to modify because tuples are immutable
            newValToList = list(newValue)

            # If probability is symbolic (not a number)
            # then replaces the symbol with its value.
            if not isinstance(value[0], numbers.Number):
                # Gets the result of the expression
                # when all symbols are replaced with their
                # numeric value. Ex) if t0_{0} = .3 then
                # '1-t0_{0}' will become 0.7
                numProb = copy.deepcopy(value[0])
                for sym in value[0].free_symbols:
                    numProb = numProb.subs(sym, probAssign[sym])

                # Creates new numeric valued probability tuple
                newValToList[0] = numProb
                newValue = tuple(newValToList)

            newScen[newKey] = newValue

        # Adds new scenario to end of list to maintain the order.
        new_scenarios.append(newScen)

    # Edges in new scenario graph will be the same adjacency list because
    # scenarios are encoded as their index

    return ScenarioGraph(new_scenarios, copy.deepcopy(scen_graph.edges), copy.deepcopy(scen_graph.weight), copy.deepcopy(scen_graph.choice_map))


# Input: a scenario graph and a state.
# Output: the index of the scenario which has the state's event in it's
# domain. If it doesn't exist an exception is raised.
def find_scenario(scen_graph, init_state):
    for i in range(0, len(scen_graph.scenarios)):
        if (init_state[2], 0) in scen_graph.scenarios[i].keys():
            return i
    raise Exception("Event does not exist in any scenario!")


def make_args(scen_graph, state, child_index, lim_cond):
  children = []

  # Stores the actual child scenario
  child_scen = scen_graph.scenarios[child_index]

  # Unpacks current state
  (score_dif, curr_team, curr_event, time_left, curr_prob) = state


  # Generates each state and recursively calls function on them.
  for (event, j), (prob, score_change, pos_change, time_dif) in child_scen.items():
    # Calculates time of child state
    child_time = time_left - time_dif
    # Sets event of child state
    child_event = event

    # Creates new child state if time is not less than 0
    # and team with possession is the team of the scenario key.
    if (child_time >= 0) and (curr_team == j):

        # If the current scenario is a choice scenario and the subsequent event is forbidden
        # by a limiting condition we continue the loop and do not generate the state.
        if (scen_graph.choice_map[child_index] != 0) and (child_event in lim_cond.forbidden_event(state)):
            continue

        # Calculates score of child state
        child_score = score_dif + ((-1) ** curr_team) * score_change

        # Calculates team of child state
        child_team = (curr_team + pos_change) % 2

        # Creates child node
        children.append((scen_graph, child_index, (child_score, child_team, child_event, child_time, prob), lim_cond))
  return children

# Input: 'self' is a ScenarioGraph, 'init_state' is a state,
# 'lim_cond' is a limiting condiiton.
# Returns the the optimal choice FOR A CHOICE STATE.
# ONLY WORKS FOR NUMERICAL scenario graphs.
def get_choice(scen_graph, init_state, lim_cond):
  # Finds the scenario of which the event of 'init_state' can be found in the domain
  scen_index = find_scenario(scen_graph, init_state)
  args = (scen_graph, scen_index, init_state, lim_cond)
  return get_choice_helper(args)



# Returns the optimal probability of winning at a state in terms of the
# children states and also a list containing the choice made if any was made at all.
def get_choice_helper(args):
  (scen_graph, scen_index, state, lim_cond) = args
  child_index = -1

  # Stores each component of the current state
  (score_dif, curr_team, curr_event, time_left, curr_prob) = state

  # If no children states then we compute the optimal probability for the state
  if not (bool(scen_graph.edges[scen_index])):
    if score_dif > 0:
      return 1, []
    elif score_dif == 0:
      return 1/2, []
    else:
      return 0, []

  # Finds which child scenario is transitioned to from the
  # current scenario when the current state's event occurs.
  for j in scen_graph.edges[scen_index]:
    if scen_graph.weight[(scen_index, j)] == curr_event:
        child_index = j
        break

  if child_index == -1:
    raise Exception("Next Scenario Not Found!")

  children = make_args(scen_graph, state, child_index, lim_cond)


  # If there are no children state generated (bc time ran out or limiting conditon)
  # then treat like end state and computes optimal probability of current node.
  # Otherwise there were children states so we assign the optimal probability of the current node
  # depending on whether the child scenario is a choice state.
  if not children:
    if score_dif > 0:
      return 1, []
    elif score_dif == 0:
      return 1/2, []
    else:
      return 0, []
  else:
    pool = multiprocessing.Pool()
    child_probs = map(lambda x: x[0], pool.map(get_choice_helper, children))

    # If node is a choice state and Team 0 makes the choice
    # returns the maximum of the optimal probability of the children
    # and the choice made.
    if ((scen_graph.choice_map[child_index] == 1) and (curr_team == 0)) or ((scen_graph.choice_map[child_index] == -1) and (curr_team == 1)):
      max_val = 0
      max_event = ''
      for i in range(len(child_probs)):
        child_prob = child_probs[i]
        if max_val < child_prob:
          max_val = child_prob
          max_event = children[i][2][2]
      return max_val, [max_event]

    # If node is a choice state and Team 0 makes the choice
    # returns the minimum of the optimal probability of the children
    # and the choice made.
    elif ((scen_graph.choice_map[child_index] == 1) and (curr_team == 1)) or ((scen_graph.choice_map[child_index] == -1) and (curr_team == 0)):
      min_val = child_probs[0]
      min_event = ''
      for i in range(len(child_probs)):
        child_prob = child_probs[i]
        if min_val > child_prob:
          min_val = child_prob
          min_event = children[i][2][2]
      return min_val, [min_event]

    # If node is a chance state assigns the expected optimal probability
    # to the current node.
    else:
      # Returns sum of expected optimal probability.
      return np.sum(np.array(map(lambda x: x[2][4], children))*np.array(child_probs)), []

In [19]:
F0 = {('start', 0): (1, 0, 0, 0), ('start', 1): (1, 0, 0, 0)}
F1 = {('foul', 0): (-1, 0, 0, 0), ('noFoul', 0): (-1, 0, 0, 0),
      ('foul', 1): (-1, 0, 0, 0), ('noFoul', 1): (-1, 0, 0, 0)}
F2 = {('2ptShot', 0): (-1, 0, 0, 0), ('3ptShot', 0): (-1, 0, 0, 0),
      ('2ptShot', 1): (-1, 0, 0, 0), ('3ptShot', 1): (-1, 0, 0, 0),
      ('hold', 0): (-1, 0, 0, 0), ('hold', 1): (-1, 0, 0, 0)}
F3 = {('turnover', 0): (Symbol('to_{0}'), 0, 1, 2), ('noTurnover', 0): (1-Symbol('to_{0}'), 0, 0, 2),
      ('turnover', 1): (Symbol('to_{1}'), 0, 1, 2), ('noTurnover', 1): (1-Symbol('to_{1}'), 0, 0, 2)}
F4 = {('made2pt', 0): (Symbol('fg^{(2)}_{0}'), 2, 1, 1), ('miss2pt', 0): (1-Symbol('fg^{(2)}_{0}'), 0, 0, 1),
       ('made2pt', 1): (Symbol('fg^{(2)}_{1}'), 2, 1, 1), ('miss2pt', 1): (1-Symbol('fg^{(2)}_{1}'), 0, 0, 1)}
F5 = {('made3pt', 0): (Symbol('fg^{(3)}_{0}'), 3, 1, 1), ('miss3pt', 0): (1-Symbol('fg^{(3)}_{0}'), 0, 0, 1),
      ('made3pt', 1): (Symbol('fg^{(3)}_{1}'), 3, 1, 1), ('miss3pt', 1): (1-Symbol('fg^{(3)}_{1}'), 0, 0, 1)}
F6 = {('offReb', 0): (Symbol('r_{0}'), 0, 0, 2), ('defReb', 0): (1-Symbol('r_{0}'), 0, 1, 2),
      ('offReb', 1): (Symbol('r_{1}'), 0, 0, 2), ('defReb', 1): (1-Symbol('r_{1}'), 0, 1, 2)}
F7 = {('madeFT1', 0): (Symbol('fg^{(1)}_{0}'), 1, 0, 1), ('missFT1', 0): (1-Symbol('fg^{(1)}_{0}'), 0, 0, 1),
      ('madeFT1', 1): (Symbol('fg^{(1)}_{1}'), 1, 0, 1), ('missFT1', 1): (1-Symbol('fg^{(1)}_{1}'), 0, 0, 1)}
F8 = {('flopFT', 0): (-1, 0, 0, 0), ('tryFT', 0): (-1, 0, 0, 0),
      ('flopFT', 1): (-1, 0, 0, 0), ('tryFT', 1): (-1, 0, 0, 0)}
F9 = {('madeFT2', 0): (Symbol('fg^{(1)}_{0}'), 1, 1, 1), ('missFT2', 0): (1-Symbol('fg^{(1)}_{0}'), 0, 0, 1),
       ('madeFT2', 1): (Symbol('fg^{(1)}_{1}'), 1, 1, 1), ('missFT2', 1): (1-Symbol('fg^{(1)}_{1}'), 0, 0, 1)}
F10 = {('x', 0): (1, 0, 0, 0), ('x', 1): (1, 0, 0, 0)}

probAssign = {Symbol('to_{0}'): 0.1053, Symbol('to_{1}'): .2,
              Symbol('fg^{(2)}_{0}'): 0.5161, Symbol('fg^{(2)}_{1}'): 0.5522,
              Symbol('fg^{(3)}_{0}'): 0.372,  Symbol('fg^{(3)}_{1}'): 0.357,
              Symbol('r_{0}'): 0.5,  Symbol('r_{1}'): 0.5,
              Symbol('fg^{(1)}_{0}'): 0.78,  Symbol('fg^{(1)}_{1}'): 0.78}

realisticModel = ScenarioGraph([F0, F1, F2, F3, F4, F5, F6, F7, F8, F9, F10],
                               {0: [1], 1: [2, 7], 2: [3, 4, 5], 3: [0, 1], 4: [1, 6], 5: [1, 6], 6: [1, 3], 7: [8, 10],
                                8: [6, 9], 9: [1, 6], 10: [8]},
                               {(0, 1): 'start',
                                (1, 2): 'noFoul', (1, 7): 'foul',
                                (2, 3): 'hold', (2, 4): '2ptShot', (2, 5): '3ptShot',
                                (3, 0): 'noTurnover', (3, 1): 'turnover',
                                (4, 1): 'made2pt', (4, 6): 'miss2pt',
                                (5, 1):  'made3pt', (5, 6): 'miss3pt',
                                (6, 1): 'defReb', (6, 3): 'offReb',
                                (7, 8): 'missFT1', (7, 10): 'madeFT1',
                                (8, 6): 'flopFT', (8, 9): 'tryFT',
                                (9, 1): 'madeFT2', (9, 6): 'missFT2',
                                (10, 8): 'x'},
                               {0: 0, 1: -1, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0,
                                8: 1, 9: 0, 10:0})

numRealModel = numericalScenario(realisticModel, probAssign)
conditions = Conditions([Condition(sympify('x < 0'), sympify('x >= 1'), ''), Condition(sympify('x > 0'), sympify('x <= 0'), '')])
LC = LimitingCondition({conditions: ['noFoul']})


print(get_choice(numRealModel, (1, 1, 'start', 6, 1), LC))

AssertionError: daemonic processes are not allowed to have children